In [84]:
import pandas as pd
import numpy as np
import re


In [85]:
# 1. Load data
df = pd.read_csv("merged_uci_brfss.csv")

In [86]:
df

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,pop_exerany2,pop_hlthpln1,pop_medcost,pop_checkup1,pop_income2,pop_educa,pop_employ1,pop_genhlth,pop_physhlth,pop_menthlth
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,0.840101,0.829130,0.173159,4.121359,5.088960,4.890070,0.518323,2.206055,2.631495,5.413802
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,0.840101,0.829130,0.173159,4.121359,5.088960,4.890070,0.518323,2.206055,2.631495,5.413802
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,0.692371,0.725267,0.241935,4.208333,2.876812,4.677837,0.481560,2.328411,2.066098,4.100917
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,0.813978,0.789928,0.142153,4.069789,7.359175,5.145103,0.853341,2.155977,2.028551,3.177574
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,0.791612,0.857530,0.116307,4.077830,7.647249,5.184517,0.881763,2.269544,2.548665,2.951452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99487,443847548,100162476,AfricanAmerican,Male,[70-80),?,1,3,7,3,...,0.662338,0.969506,0.073914,4.129870,5.082136,4.480046,0.134175,3.146560,5.420077,2.380398
99488,443847782,74694222,AfricanAmerican,Female,[80-90),?,1,4,5,5,...,0.594004,0.973079,0.086579,4.228261,3.171156,4.546854,0.089715,3.182330,6.280530,2.732649
99489,443854148,41088789,Caucasian,Male,[70-80),?,1,1,7,1,...,0.725325,0.985508,0.031238,4.117647,7.523126,5.033737,0.191577,2.728003,4.713654,1.750391
99490,443857166,31693671,Caucasian,Female,[80-90),?,2,3,7,10,...,0.661157,0.990545,0.039166,4.172391,6.195676,4.797186,0.128604,2.713066,5.173115,2.342723


In [87]:
print("Shape:", df.shape)
print("\nColumn dtypes:\n", df.dtypes)

Shape: (99492, 73)

Column dtypes:
 encounter_id      int64
patient_nbr       int64
race             object
gender           object
age              object
                 ...   
pop_educa       float64
pop_employ1     float64
pop_genhlth     float64
pop_physhlth    float64
pop_menthlth    float64
Length: 73, dtype: object


In [88]:
# Exact row duplicates
print("Exact duplicates:", df.duplicated().sum())

# Check if encounter_id is unique
if "encounter_id" in df.columns:
    print("Unique encounter_id:", df["encounter_id"].is_unique)

# How many encounters per patient?
if "patient_nbr" in df.columns:
    print(df["patient_nbr"].value_counts().describe())


Exact duplicates: 0
Unique encounter_id: True
count    69667.000000
mean         1.428108
std          1.099823
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max         40.000000
Name: count, dtype: float64


In [89]:
# 2. Basic null check for all columns
null_counts = df.isna().sum()
null_percent = (df.isna().sum() / len(df)) * 100

print("\n=== Null values per column ===")
for col in df.columns:
    print(f"{col:25s}  nulls: {null_counts[col]:6d}  ({null_percent[col]:6.2f}%)")



=== Null values per column ===
encounter_id               nulls:      0  (  0.00%)
patient_nbr                nulls:      0  (  0.00%)
race                       nulls:      0  (  0.00%)
gender                     nulls:      0  (  0.00%)
age                        nulls:      0  (  0.00%)
weight                     nulls:      0  (  0.00%)
admission_type_id          nulls:      0  (  0.00%)
discharge_disposition_id   nulls:      0  (  0.00%)
admission_source_id        nulls:      0  (  0.00%)
time_in_hospital           nulls:      0  (  0.00%)
payer_code                 nulls:      0  (  0.00%)
medical_specialty          nulls:      0  (  0.00%)
num_lab_procedures         nulls:      0  (  0.00%)
num_procedures             nulls:      0  (  0.00%)
num_medications            nulls:      0  (  0.00%)
number_outpatient          nulls:      0  (  0.00%)
number_emergency           nulls:      0  (  0.00%)
number_inpatient           nulls:      0  (  0.00%)
diag_1                     nulls

In [90]:
# Turn all "?" into proper missing values
df = df.replace("?", np.nan)

cat_cols = [
    "race",
    "gender",
    "payer_code",
    "medical_specialty",
    "age",
    "weight"
]

for c in cat_cols:
    if c in df.columns:
        df[c] = df[c].fillna("Unknown")

In [91]:

# Replace missing values with "Not tested"
df['max_glu_serum'] = df['max_glu_serum'].replace({np.nan: 'Not tested'})
df['A1Cresult'] = df['A1Cresult'].replace({np.nan: 'Not tested'})

# Create binary flags
df['max_glu_serum_flag'] = np.where(df['max_glu_serum'] != 'Not tested', 1, 0)
df['A1Cresult_flag'] = np.where(df['A1Cresult'] != 'Not tested', 1, 0)


max_glu_serum:

What it is:
The maximum serum glucose (blood sugar) value for the patient during that hospital encounter.

How it’s stored in this dataset:
It’s not the exact number. It’s bucketed into:

"None" → no glucose test result recorded

"Norm" → glucose result normal

">200" → max glucose > 200 mg/dL

">300" → max glucose > 300 mg/dL

Why it matters:
Very high glucose during admission can indicate:

Poor diabetes control

Acute stress / illness

Higher risk of complications
It’s often used as a risk factor for worse outcomes or readmission.

A1Cresult

What it is:
Result of the Hemoglobin A1c (HbA1c) test.
That’s a lab test that measures average blood glucose over ~3 months, not just one moment.

How it’s stored:
Again, bucketed:

"None" → no A1c test result available

"Norm" → normal A1c (good long-term control)

">7" → A1c above 7%

">8" → A1c above 8%

Why it matters:
HbA1c is basically:

“How well has this person’s diabetes been controlled over the last few months?”

Higher A1c:

Worse long-term control

Higher risk of complications

Often associated with higher chance of readmission / more hospital use

In [92]:
num_cols = [
    "time_in_hospital",
    "num_lab_procedures",
    "num_procedures",
    "num_medications",
    "number_outpatient",
    "number_emergency",
    "number_inpatient"
]

for c in num_cols:
    if c in df.columns:
        df[c] = df[c].fillna(df[c].median())


In [93]:
# --- Targeted fix for diag_2 / diag_3 ---
import numpy as np
import pandas as pd

target_cols = ["diag_2", "diag_3"]

# 1) normalize blank-like tokens to NaN
for c in target_cols:
    if c in df.columns:
        # keep dtype object for predictable fills
        df[c] = df[c].astype("string")
        df[c] = (
            df[c].str.strip()
                  .replace({"": np.nan, "?": np.nan, "NA": np.nan, "N/A": np.nan,
                            "na": np.nan, "null": np.nan, "NULL": np.nan, "None": np.nan})
        )

# 2) add missingness flags BEFORE filling
for c in target_cols:
    if c in df.columns and (c + "_was_missing") not in df.columns:
        df[c + "_was_missing"] = df[c].isna().astype("Int8")

# 3) fill with explicit category so downstream is stable
for c in target_cols:
    if c in df.columns:
        df[c] = df[c].fillna("Unknown")


In [94]:
# ---------- 4. Normalize categorical text ----------
# Gender: standardize to 'male', 'female', 'other'
if "gender" in df.columns:
    df["gender"] = (
        df["gender"]
        .astype(str)
        .str.strip()
        .str.lower()
        .replace({
            "m": "male",
            "male": "male",
            "f": "female",
            "female": "female"
        })
    )
    df["gender"] = df["gender"].where(df["gender"].isin(["male", "female"]), "other")

# Race: clean weird values
if "race" in df.columns:
    df["race"] = (
        df["race"]
        .astype(str)
        .str.strip()
        .str.title()  # e.g. 'caucasian' -> 'Caucasian'
        .replace({
            "?": np.nan,
            "Unknown/Invalid": np.nan
        })
    )

# Readmitted: standardize + optional numeric label
if "readmitted" in df.columns:
    df["readmitted"] = (
        df["readmitted"]
        .astype(str)
        .str.strip()
        .str.upper()
    )
    readmit_map = {"NO": "No", "<30": "<30", ">30": ">30"}
    df["readmitted"] = df["readmitted"].map(readmit_map)

    readmit_num_map = {"No": 0, ">30": 1, "<30": 2}
    df["readmitted_label"] = df["readmitted"].map(readmit_num_map)

#Encode basic yes/no flags (diabetesMed, change)
flag_cols = ["diabetesMed", "change"]

for c in flag_cols:
    if c in df.columns:
        df[c] = (
            df[c]
            .astype(str)
            .str.strip()
            .str.lower()
            .replace({"yes": 1, "no": 0})
        )


/var/folders/p9/ylmb3zds11920sj6n2wt4s6w0000gn/T/ipykernel_43637/1531416565.py:55: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({"yes": 1, "no": 0})


Readmitted: 
"<30" → the patient was readmitted within 30 days of discharge
">30" → the patient was readmitted after 30 days
"NO" → the patient was not readmitted at all in the observation window

In [95]:
# Discharge instability score:
# Combines urgent admission, high procedure count, and short hospital stay
# to approximate how unstable the discharge was. Higher scores = higher
# risk of early readmission.

# 1) Urgent / emergency admission flag
if "admission_type_id" in df.columns:
    df["urgent_admit_flag"] = df["admission_type_id"].isin([2, 3]).astype(int)

# 2) High procedures relative to the dataset
if "num_procedures" in df.columns:
    proc_median = df["num_procedures"].median()
    df["high_procedures_flag"] = (df["num_procedures"] > proc_median).astype(int)

# 3) Short stay flag (<= 3 days)
if "time_in_hospital" in df.columns:
    df["short_stay_flag"] = (df["time_in_hospital"] <= 3).astype(int)

# 4) Discharge instability score = sum of relevant flags
instability_parts = ["urgent_admit_flag", "high_procedures_flag", "short_stay_flag"]
present_cols = [c for c in instability_parts if c in df.columns]
df["discharge_instability_score"] = df[present_cols].sum(axis=1)


In [96]:
#target
# Binary target: readmitted within 30 days (1) vs not (0)
if "readmitted" in df.columns:
    df["readmitted_30d"] = df["readmitted"].map({
        "<30": 1,
        ">30": 0,
        "No": 0
    })
    


In [97]:
df

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,max_glu_serum_flag,A1Cresult_flag,diag_2_was_missing,diag_3_was_missing,readmitted_label,urgent_admit_flag,high_procedures_flag,short_stay_flag,discharge_instability_score,readmitted_30d
0,2278392,8222157,Caucasian,female,[0-10),Unknown,6,25,1,1,...,0,0,1,1,0,0,0,1,1,0
1,149190,55629189,Caucasian,female,[10-20),Unknown,1,1,7,3,...,0,0,0,0,1,0,0,1,1,0
2,64410,86047875,Africanamerican,female,[20-30),Unknown,1,1,7,2,...,0,0,0,0,0,0,1,1,2,0
3,500364,82442376,Caucasian,male,[30-40),Unknown,1,1,7,2,...,0,0,0,0,0,0,0,1,1,0
4,16680,42519267,Caucasian,male,[40-50),Unknown,1,1,7,1,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99487,443847548,100162476,Africanamerican,male,[70-80),Unknown,1,3,7,3,...,0,1,0,0,1,0,0,1,1,0
99488,443847782,74694222,Africanamerican,female,[80-90),Unknown,1,4,5,5,...,0,0,0,0,0,0,1,0,1,0
99489,443854148,41088789,Caucasian,male,[70-80),Unknown,1,1,7,1,...,0,0,0,0,0,0,0,1,1,0
99490,443857166,31693671,Caucasian,female,[80-90),Unknown,2,3,7,10,...,0,0,0,0,0,1,1,0,2,0


In [98]:
missing = (
    df.isna().sum()
      .to_frame("missing_count")
      .assign(missing_pct=lambda x: (x["missing_count"]/len(df)).round(3))
      .sort_values("missing_pct", ascending=False)
)
print(missing.head(20))


                          missing_count  missing_pct
encounter_id                          0          0.0
pop_smoke100                          0          0.0
pop_addepev2                          0          0.0
pop_havarth3                          0          0.0
pop_chccopd1                          0          0.0
pop_asthma3                           0          0.0
pop_toldhi2                           0          0.0
pop_bphigh4                           0          0.0
pop_cvdstrk3                          0          0.0
pop_cvdinfr4                          0          0.0
pop_diabete3                          0          0.0
demo_key                              0          0.0
age_mapped                            0          0.0
readmitted                            0          0.0
diabetesMed                           0          0.0
change                                0          0.0
metformin-pioglitazone                0          0.0
metformin-rosiglitazone               0       

In [99]:
import os
import pandas as pd

# --- PICK THE RIGHT DATAFRAME ---
# If your cleaned frame is df_clean, use that. If you used excel_safe(...), use df_out.
to_save = df_out  # or: to_save = df_clean

# --- CSV SETTINGS ---
out_csv = "cleaned_dataset.csv"

# Make sure dates are plain strings (optional but nice)
date_like = [c for c in to_save.columns if c.lower().endswith(("date","_dt","_time"))]
if date_like:
    tmp = to_save.copy()
    for c in date_like:
        tmp[c] = pd.to_datetime(tmp[c], errors="coerce").dt.tz_localize(None)
        tmp[c] = tmp[c].astype("datetime64[ns]").astype(str)
else:
    tmp = to_save

# Write CSV only (fast, robust). Adjust NA representation if you want blanks instead of "NaN".
tmp.to_csv(
    out_csv,
    index=False,
    encoding="utf-8",
    na_rep="",          # write missing as empty cells; change to "NA" if you prefer
    quoting=1           # csv.QUOTE_ALL to be Excel-friendly with commas
)

print(f"Saved CSV → {out_csv}")
print("Exists?", os.path.exists(out_csv))
print("Rows x Cols:", tmp.shape)


Saved CSV → cleaned_dataset.csv
Exists? True
Rows x Cols: (99492, 83)
